##SetFit

🤗 SetFit is an efficient and prompt-free framework for few-shot fine-tuning of Sentence Transformers. It achieves high accuracy with little labeled data - for instance, with only 8 labeled examples per class on the Customer Reviews sentiment dataset, 🤗 SetFit is competitive with fine-tuning RoBERTa Large on the full training set of 3k examples! See all details here
https://huggingface.co/docs/setfit/index

## Task
Using the SetFit library, we will build a sentence classifier to classify inclusive language-related sentences depending on the context.

**Note!** Please be sure to use GPU to train. You can check and change at "Runtime" tab above

## Setup

In [1]:
#!pip install --upgrade transformers

In [2]:
!pip install setfit transformers==4.39.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/

In [3]:
!pip3 install setfit -q

First, you have to store your authentication token from the Hugging Face Hub  https://huggingface.co/ (sign up here if you haven't already!). To do so, execute the following cell and input an access token associated with your account:

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Loading and sampling the dataset

We load dataset from your repo at HuggingFace account

In [5]:
#Load dataset
from datasets import load_dataset, Dataset

dataset = load_dataset("oknerazan/english_sentences", use_auth_token=True)
dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/310 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence', 'purpose', 'word'],
        num_rows: 310
    })
})

convert dataset to pandas to explore

In [6]:
df = dataset["train"].to_pandas()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   label     310 non-null    int64 
 1   sentence  310 non-null    object
 2   purpose   310 non-null    object
 3   word      310 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.8+ KB


Split data into train and test sets

In [8]:
#train data
df_train = df[df["purpose"]=="train"]
df_train = df_train[["label", "sentence", "word"]]
#create a subset of dataframe
df_train=df_train.loc[:, 'label':'sentence']
df_train = df_train.reset_index(drop=True)
df_train = Dataset.from_pandas(df_train)

In [9]:
# test data
df_test = df[df["purpose"]=="test"]
df_test = df_test[["label", "sentence", "word"]]
#create a subset of dataframe
df_test = df_test.loc[:, 'label':'sentence']
df_test = df_test.reset_index(drop=True)
df_test = Dataset.from_pandas(df_test)

Explore the train and test sets

In [10]:
df_train

Dataset({
    features: ['label', 'sentence'],
    num_rows: 186
})

In [11]:
df_test

Dataset({
    features: ['label', 'sentence'],
    num_rows: 66
})

We are ready to train the model

## Fine-tuning the model

To train a SetFit model, the first thing to do is download a pretrained checkpoint from the Hub. We can do so by using the from_pretrained() method associated with the SetFitModel class:

Change the values below to try a different models! Try at least three different models
For example, try as well sentence-transformers/paraphrase-mpnet-base-v2

Here you can explore all avalible pretrained sentence-transformers models
https://huggingface.co/models?library=sentence-transformers&sort=downloads

In [12]:
from setfit import SetFitModel
model_id = "sentence-transformers/all-mpnet-base-v2"
model = SetFitModel.from_pretrained(model_id)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Here, we've downloaded a pretrained Sentence Transformer from the Hub and added a logistic classification head to the create the SetFit model. As indicated in the message, we need to train this model on some labeled examples. We can do so by using the SetFitTrainer class as follows:

In [13]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=df_train,
    eval_dataset=df_test,
    loss_class=CosineSimilarityLoss,
    batch_size=16, #reduce to 8
    num_epochs=1,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

<ipython-input-13-a28eda21eaef>:4: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/186 [00:00<?, ? examples/s]

The main arguments to notice in the trainer is the following:
loss_class: The loss function to use for contrastive learning with the Sentence Transformer body
num_iterations: The number of text pairs to generate for contrastive learning
column_mapping: The SetFitTrainer expects the inputs to be found in a text and label column. This mapping automatically formats the training and evaluation datasets for us.

Now that we've created a trainer, we can train it!


In [ ]:
trainer.train()

## Metrics and evaluation

The final step is to compute the model's performance using the evaluate() method:

In [ ]:
metrics = trainer.evaluate()

***** Running evaluation *****


check metrics

In [ ]:
metrics

{'accuracy': 0.9393939393939394}

Try to train the different models and compare the metrics

## Predictions and tests

Below are sentence samples that you can use and run to see if your classifier can catch the nuances of the language to distinguish between two synonyms.

"you look like an old fossyl", "Fossil fuels is an old source of energy", "you will have a flexible schedule", "you need to have a flexible personality", "you should give a valuable impact", "We have the latest styles & trends of Fossil watches, wallets, bags and accessories.", "FREE Shipping & Returns at Fossil.com." "a flexible substance or material, as rubber or leather." "The impact of water pollution on aquatic life and land life can be devastating."


In [ ]:
# Run inference
preds = model(["you look like an old fossyl", "Fossil fuels is an old source of energy", "you will have a flexible schedule", "you need to have a flexible personality", "you should give a valuable impact"])
preds

tensor([1, 0, 0, 1, 1])

In [ ]:
preds = model(["A simple black spider is a fast face painting design that can make a big impact come Halloween."])

In [ ]:
preds = model(["you should keep your schedule flexible"])
for item in preds:
  if item == 1:
    print ("highlight")
  else:
    print ("false positive")

highlight
